In [ ]:
run_cycle = 1

In [ ]:
import random
import datetime
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

import numpy as np

import csv
import os

import sequence.utils.log as log

def write_to_csv(file_path, data):
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

now = datetime.datetime.now()
date_time_str =now.strftime('%Y%m%d_%H%M%S')


def test(sim_time, cc_delay, qc_atten, qc_dist, num_nodes, starting_node, desired_node):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ms)
    qc_atten: attenuation on quantum channels (dB/m)
    qc_dist: distance of quantum channels (km) 
    num_nodes: number of router nodes you want the system to have
    """
    
    PS_PER_MS = 1e9
    M_PER_KM = 1e3
    
    qc_dist = qc_dist.astype(float)
    # convert units for cc delay (to ps) and qc distance (to m)
    cc_delay *= PS_PER_MS
    qc_dist *= M_PER_KM
    
    raw_fidelity = 0.75
    
    log_filename = "multi_node_framework_log " + str(run_cycle) + ".log"
    
    # construct the simulation timeline; the constructor argument is the simulation time (in ps)
    tl = Timeline(sim_time * PS_PER_MS)
    tl.show_progress = True
    
    log.set_logger(__name__, tl, log_filename)
    log.set_logger_level('INFO')
    #log.track_module('timeline')
    #log.track_module('network_manager')
    log.track_module('generation')
    #log.track_module('node')
    
    # DO THIS IN A LOOP, HOW DO THE MEMORIES WORK THOUGH?, 
    # MAKE ROUTERS AN ARRAY AND PASS IN ANOTHER PARAMETER WHICH SPECIFIES NUMBER OF NODES
    routers = [QuantumRouter] * num_nodes
    ## create our quantum network and update parameters as needed
    for i in range(0, num_nodes):
        routers[i] = QuantumRouter("r" + str(i), tl, 100)
        
    BSMNodes = [BSMNode] * (num_nodes - 1)
    for i in range(0, num_nodes - 1):
        BSMNodes[i] = BSMNode("m" + str(i), tl, [routers[i].name, routers[i+1].name])    

    for i in range(0,num_nodes - 1):
        routers[i].add_bsm_node(BSMNodes[i].name, routers[i + 1].name)
        routers[i + 1].add_bsm_node(BSMNodes[i].name, routers[i].name)
    
    # set seeds for random generators
    nodes = []
    nodes_seeds = []
    for router in routers:
        nodes.append(router)
    for bsm_node in BSMNodes:
        nodes.append(bsm_node)
    for i, node in enumerate(nodes):
        seed_random = random.randint(1, len(nodes))
        node.set_seed(seed_random)
        nodes_seeds.append(seed_random)
        #print(node.name,seed_random)
    
    for node in routers:
        memory_array = node.get_components_by_type("MemoryArray")[0]
        # we update the coherence time (measured in seconds) here
        memory_array.update_memory_params("coherence_time", 10)
        # and similarly update the fidelity of entanglement for the memories
        memory_array.update_memory_params("raw_fidelity", raw_fidelity)
    
    # create all-to-all classical connections
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            # construct a classical communication channel
            # (with arguments for the channel name, timeline, length (in m), and delay (in ps))
            cc = ClassicalChannel("cc_%s_%s"%(node1.name, node2.name), tl, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2.name)
            
    # NOTE: CLASSICAL CHANNEL DELAY CAN NOT CURRENTLY BE DIFFERENT FOR EACH CLASSICAL CHANNEL

    
    # DO THIS IN A LOOP, TAKE IN qc_dist, qc_atten as arrays 
            
    # create quantum channels linking r1 and r2 to m1
    # (with arguments for the channel name, timeline, attenuation (in dB/km), and distance (in m))
    qc_array = [None] * (num_nodes - 1) * 2
    j = 0
    for i in range(0, len(qc_array), 2):
        qc_array[i] = QuantumChannel("qc_r" + str(j) + "_m" + str(j), tl, qc_atten, qc_dist[i])
        qc_array[i + 1] = QuantumChannel("qc_r" + str(j + 1) + "_m" + str(j), tl, qc_atten, qc_dist[i + 1])
        qc_array[i].set_ends(routers[j], BSMNodes[j].name)
        qc_array[i + 1].set_ends(routers[j + 1], BSMNodes[j].name)
        j += 1

    # routing table setup and forwarding rules
    for i in range(len(routers)):
        for j in range(len(routers)):
            if j != i:
                if i > j:
                    routers[i].network_manager.protocol_stack[0].add_forwarding_rule("r" + str(j), "r" + str(i-1))
                if i < j:
                    routers[i].network_manager.protocol_stack[0].add_forwarding_rule("r" + str(j), "r" + str(i+1))
    
    ## run our simulation
    
    tl.init()
    # we use the network manager of an end router to make our entanglement request
    # here, the arguments are:
    # (1) the destination node name,
    # (2) the start time (in ps) of entanglement,
    # (3) the end time (in ps) of entanglement,
    # (4) the number of memories to entangle, and
    # (5) the desired fidelity of entanglement.
    routers[starting_node].network_manager.request("r" + str(desired_node), 1e12, 1e14, 20, 0.9)

    tick = time.time()
    tl.run()
    #print("execution time %.2f sec" % (time.time() - tick))

    routers_to_plot = [routers[starting_node],routers[desired_node]]
    
    csv_file_path = "/home/uom/howe0427/SeQUeNCe/plots/test_data.csv"
    
    file_exists = os.path.isfile(csv_file_path) and os.path.getsize(csv_file_path) > 0
    with open(csv_file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(headers)
        for i, router in enumerate(routers_to_plot):
            for info in router.resource_manager.memory_manager:
                if info.entangle_time > 0:
                    entangle_time = info.entangle_time / 1e12
                    fidelity = info.fidelity
                    row_data = [router.name, nodes_seeds[i], entangle_time, fidelity]
                    writer.writerow(row_data)
    #print(tl.run_counter)
    #print(tl.schedule_counter)
    #print(tl.entities)
    #print(tl.events)
#interactive_plot = interact(test, sim_time=(2000, 4000, 500), cc_delay=(0.1, 1, 0.1), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
#interactive_plot
num_nodes = 64
size = 2*(num_nodes - 1)
#qc_distances = np.random.randint(1, 9, size)
tot_distance = 1000
qc_distances = np.ones(size) * (tot_distance / size)
#qc_distances = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
#print(qc_distances)
starting = 0
desired = 63
test(200000000, 0.1, 1e-5, qc_distances, num_nodes, starting, desired)

[55.55555556 55.55555556 55.55555556 55.55555556 55.55555556 55.55555556
 55.55555556 55.55555556 55.55555556 55.55555556 55.55555556 55.55555556
 55.55555556 55.55555556 55.55555556 55.55555556 55.55555556 55.55555556]
r0 15
r1 4
r2 10
r3 1
r4 4
r5 18
r6 10
r7 4
r8 4
r9 9
m0 7
m1 10
m2 14
m3 4
m4 16
m5 4
m6 3
m7 9
m8 3


KeyboardInterrupt: 

### Results

We note that the number of entangled memories increses exponentially over time. This is due to memory expiration, with a coherence time of 0.3 seconds, as well as the consumption of memories for purification and swapping soon after they are entangled. We also see distinct layers of fidelities, reflecting the number of times an individual memory has been purified or swapped.